In [1]:
import torch

In [2]:
from config import Config
import gym

In [3]:
from utils.replay_memory import ReplayMemory, Transition
from utils.reproducibility import set_seed
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from collections import deque
from Networks.LaplaceDQN_net import * 
from Networks.LaplaceDQN_net_monot_multgam import *
from utils.Inverse_Laplace import SVD_approximation_inverse_Laplace
from itertools import count
import random
import math
import torch
import os

import gym
from config import Config

import time

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
class DQNNetMultgam(nn.Module):

    def __init__(self, config):
        self.config = config
        self.input_dim = config.input_dim + 1 # add one for the discount factor 
        self.action_dim = config.action_dim
        self.num_sensitivities = config.num_sensitivities
        output_dim = self.action_dim * self.num_sensitivities

        super(DQNNetMultgam, self).__init__()
        self.layer1 = nn.Linear(self.input_dim, 64) # originally 128
        self.layer2 = nn.Linear(64, 64)
        self.layer3 = nn.Linear(64, output_dim)
        self.reshape_layer = lambda x: x.view(-1, self.action_dim, self.num_sensitivities)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x)) 
        x = self.reshape_layer(x)
        
        x = torch.cumsum(x, dim=2)
        x = torch.flip(x, dims=[2])
                
        diffs = torch.diff(x, dim=2)
        #print(x.shape, diffs.shape, x, diffs)
        # assert x == diffs
        assert torch.all(diffs <= 0), "Output Tensor is not non-increasing along dimension 2"
        
        return x

In [6]:
class DQNNetMultgamInv(nn.Module):

    def __init__(self, config):
        self.config = config
        self.input_dim = config.input_dim + 1 # add one for the discount factor 
        self.action_dim = config.action_dim
        self.num_sensitivities = config.num_sensitivities
        output_dim = self.action_dim * self.num_sensitivities

        super(DQNNetMultgamInv, self).__init__()
        self.low_net = DQNNetMultgam(config)
        self.med_net = DQNNetMultgam(config)
        self.high_net = DQNNetMultgam(config)

        self.low_range = 0.5
        self.med_range = 0.75
        self.high_range = 1.0
        

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        # x: [B, num_states + 1(gamma-dim)]
        gamm_tens = x[:,-1]
        low_gam_mask = (gamm_tens >= 0.0) & (gamm_tens < 0.5)
        med_gam_mask = (gamm_tens >= 0.5) &  (gamm_tens < 0.75)
        high_gam_mask = (gamm_tens >= 0.75) &  (gamm_tens <= 1)

        # assumed gammas in range [0,1]
        assert (low_gam_mask.sum()+med_gam_mask.sum()+high_gam_mask.sum()) == gamm_tens.shape[0]

        # Collect the outputs from each network
        low_out = self.low_net(x[low_gam_mask])
        med_out = self.med_net(x[med_gam_mask])
        high_out = self.high_net(x[high_gam_mask])

        # Initialize the result tensor with the same dtype and device as input x
        res_tens = torch.zeros(x.shape[0], self.action_dim, self.num_sensitivities, dtype=x.dtype, device=x.device)

        # Indexes for placing outputs in the result tensor
        low_indexes = low_gam_mask.nonzero(as_tuple=True)[0]
        med_indexes = med_gam_mask.nonzero(as_tuple=True)[0]
        high_indexes = high_gam_mask.nonzero(as_tuple=True)[0]

        # Place the outputs in the correct positions
        res_tens[low_indexes] = low_out
        res_tens[med_indexes] = med_out
        res_tens[high_indexes] = high_out
        
        return self.low_net(x)

In [7]:
game = 'CartPole-v1'
env = gym.make(game, None) 
config = Config(env)

In [8]:
class LaplaceDQNAgentMultgamIntv:

    def __init__(self, game, config):
        # env = gym.make(game, None) 
        # config = Config(env)
        self.config = config

        self.envs = [gym.make(game, None) for _ in range(config.num_gamma)]
        # self.envs = [env] + [gym.make(game, None) for _ in range(config.num_gamma-1)]

        for env in self.envs:
            env.action_space.seed(config.seed)
        
        self.config = config 
        self.input_dim = config.input_dim
        self.action_dim = config.action_dim

        self.num_episodes = config.num_episodes
        self.initial_number_steps = config.initial_number_steps
        self.later_number_steps = config.later_number_steps
        self.max_steps = self.initial_number_steps
        self.BATCH_SIZE = config.BATCH_SIZE
        self.LR = config.LR
        self.TAU = config.TAU
        self.device = config.device
        
        self.num_gamma = config.num_gamma 
        self.gamma_min = config.gamma_min
        self.gamma_max = config.gamma_max
        start = 1 / np.log(self.gamma_min) 
        #start = 1 / np.log(0.99) # with 1 gamma TODO
        end = 1 / np.log(self.gamma_max)
        #end = 1 / np.log(0.95)   
        self.gammas = torch.exp(torch.true_divide(1, torch.linspace(start, end, self.num_gamma)))
        self.num_sensitivities = config.num_sensitivities 
        self.rmin = config.rmin
        self.rmax = config.rmax
        assert self.rmin < self.rmax, "The reward range is not valid"
        self.sensitivity_step = (self.rmax - self.rmin) / self.num_sensitivities
        self.sensitivities = torch.arange(self.rmin, self.rmax, self.sensitivity_step)
        self.middle_sensitivities = torch.tensor([torch.true_divide(self.sensitivities[i] + self.sensitivities[i+1], 2) for i in range(self.sensitivities.shape[0]-1)])
        self.activ_sharpness = config.activ_sharpness
        
        self.num_gamma_to_tau = config.num_gamma_to_tau
        self.gamma_to_tau_min = config.gamma_to_tau_min
        self.gamma_to_tau_max = config.gamma_to_tau_max
        start = 1 / np.log(self.gamma_to_tau_min) 
        end = 1 / np.log(self.gamma_to_tau_max)   
        self.gammas_to_tau = torch.exp(torch.true_divide(1, torch.linspace(start, end, self.num_gamma_to_tau)))
        self.K = config.K
        self.time_horizon_change = config.time_horizon_change
        
        # taus=torch.linspace(0.01,3,self.num_gamma) # as in implementation from Pablo Tano
        # self.gammas=torch.exp(-1/taus)
        print("Gammas : ", self.gammas)
        
        self.total_steps = [0 for _ in range(self.num_gamma)]

        # reproducibility
        self.seed = config.seed
        set_seed(self.seed)
        
        self.dir = os.getcwd()

        # copying weights of base_net to policy_net and target_net
        self.policy_net = DQNNetMultgamInv(self.config) 
        self.target_net = DQNNetMultgamInv(self.config)
        #self.policy_net = DQNNetMultgam(self.config)
        #self.target_net = DQNNetMultgam(self.config)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.optimizer = optim.AdamW(self.policy_net.parameters(), lr=self.LR, amsgrad=True)
        self.criterion = nn.SmoothL1Loss()

        self.replay_buffer_size = config.replay_buffer_size
        self.replay_memory = [ReplayMemory(self.replay_buffer_size) for _ in range(self.num_gamma)] 
        
        self.episode_durations = []
        self.check_model_improved = [torch.tensor([0]) for _ in range(self.num_gamma)]
        self.best_max = [torch.tensor([0]) for _ in range(self.num_gamma)]

        # for select action (epsilon-greedy)
        self.steps_done = [0 for _ in range(self.num_gamma)]
        
        # save for plotting evolution during training for each gamma
        self.model_reward_hist = [[] for _ in range(self.num_gamma)]
        self.model_loss_hist = [[] for _ in range(self.num_gamma)]
    
    
    def transition(self):
        """
        In transition, the agent simply plays and records
        [current_state, action, reward, next_state, done]
        in the replay_memory

        Updating the weights of the neural network happens
        every single time the replay buffer size is reached.

        done: boolean, whether the game has ended or not.
        """
        # Start time
        self.start_time = time.time()
        
        # multiple environments for every gamma
        self.states = [torch.from_numpy(env.reset(seed=self.seed)[0]).float().unsqueeze(0) for env in self.envs] 
            
        for i_episode in range(self.num_episodes):
            # No horizon change during training for now
            # if self.time_horizon_change is not None and i_episode == self.time_horizon_change:
            #     self.max_steps = self.later_number_steps

            for gamma_idx, gamma in enumerate(self.gammas):
                print('Episode: {} Reward: {} Max_Reward: {}'.format(i_episode, self.check_model_improved[gamma_idx].item(), self.best_max[gamma_idx].item()))
                print('-' * 64)
                self.check_model_improved[gamma_idx] = 0
                for t in count():    
                    env = self.envs[gamma_idx]
                    state = self.states[gamma_idx]
                    
                    action = self.select_action(gamma_idx)
                    # No horizon change during training for now
                    # if (self.time_horizon_change is None) or (self.time_horizon_change is not None and i_episode < self.time_horizon_change):
                    #     action = self.select_action(gamma_idx)
                    # else:
                    #     action = self.select_action_after_H_change(gamma_idx) 
                        
                    observation, reward, terminated, truncated, _ = env.step(action.item())
                    reward = torch.tensor([reward], device=self.device)
                    done = terminated or truncated

                    # if terminated or t == self.max_steps:
                    if terminated:
                        next_state = None
                    else:
                        next_state = torch.tensor(observation, dtype=torch.float32, device=self.device).unsqueeze(0)

                    # Store the transition in memory
                    self.replay_memory[gamma_idx].push(state, action, next_state, reward)

                    # Move to the next state
                    self.states[gamma_idx] = next_state
                    self.total_steps[gamma_idx] += 1

                    # Perform one step of the optimization (on the policy network)
                    self.train_by_replay(gamma_idx) 

                    # Soft update of the target network's weights 
                    # θ′ ← τ θ + (1 −τ )θ′
                    # previous implementation updates were done for any episode where the reward is higher
                    target_net_state_dict = self.target_net.state_dict()
                    policy_net_state_dict = self.policy_net.state_dict()
                    for key in policy_net_state_dict:
                        target_net_state_dict[key] = policy_net_state_dict[key]*self.TAU + target_net_state_dict[key]*(1-self.TAU)
                    self.target_net.load_state_dict(target_net_state_dict)
                    
                    self.check_model_improved[gamma_idx] += reward 
                    # if done or t == self.max_steps:
                    if done:
                        self.episode_durations.append(t + 1)
                        self.model_reward_hist[gamma_idx].append(self.check_model_improved[gamma_idx].detach().numpy())
                        self.states[gamma_idx] = torch.from_numpy(self.envs[gamma_idx].reset(seed=self.seed)[0]).float().unsqueeze(0)
                        break 

                if self.check_model_improved[gamma_idx] > self.best_max[gamma_idx]:
                    self.best_max[gamma_idx] = self.check_model_improved[gamma_idx]
                    
        folder = 'Results'
        subfolder_reward = 'Rewards'
        subfolder_loss = "Losses"
        subfolder_weights = "Weights"
        filename_rewards = 'rewards_laplace_mult.npy'
        filename_losses = 'losses_laplace_mult.npy'
        filename_weights = 'policy_net_weights_laplace_mult.pth'
        path_rewards = os.path.join(self.dir, folder, subfolder_reward, filename_rewards)
        path_losses = os.path.join(self.dir, folder, subfolder_loss, filename_losses)
        path_weights = os.path.join(self.dir, folder, subfolder_weights, filename_weights)
                
        # Save the losses and rewards to numpy arrays
        cum_reward_per_episode = np.array([self.model_reward_hist[i] for i in range(self.num_gamma)])
        np.save(path_rewards, cum_reward_per_episode)
        
        # arrange dimension difference due to difference transition lengths
        # append -1, to concatenate them, since loss cannot be -1
        vectors = self.model_loss_hist
        max_length = max(len(vec) for vec in vectors)
        padded_vectors = [np.pad(vec, (0, max_length - len(vec)), 'constant', constant_values=-1) for vec in vectors]
        loss_result = np.vstack(padded_vectors)
        np.save(path_losses, np.array(loss_result)) # check if I can save a numpy array of lists

        torch.save(self.policy_net.state_dict(), path_weights)
        
        # End time
        self.end_time = time.time()
        
        
    def train_by_replay(self, gamma_idx):
        """
        TD update by replaying the history.
        """
        # step 1: generate replay samples (size = self.batch_size) from the replay buffer
        # e.g. uniform random replay or prioritize experience replay
        if len(self.replay_memory[gamma_idx]) < self.BATCH_SIZE:
            return
        transitions = self.replay_memory[gamma_idx].sample(self.BATCH_SIZE)

        # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
        # detailed explanation). This converts batch-array of Transitions
        # to Transition of batch-arrays.
        batch = Transition(*zip(*transitions))

        # Compute a mask of non-final states and concatenate the batch elements
        # (a final state would've been the one after which simulation ended)
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                            batch.next_state)), device=self.device, dtype=torch.bool)
        non_final_next_states = torch.cat([s for s in batch.next_state
                                                    if s is not None])
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        
        # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
        # columns of actions taken. These are the actions which would've been taken
        # for each batch state according to policy_net
        input = torch.cat((state_batch, torch.tensor([self.gammas[gamma_idx]]).unsqueeze(0).repeat(state_batch.shape[0], 1)), dim=1)  
        assert state_batch.shape[0] == input.shape[0]
        output = self.policy_net(input) 
        state_action_values = output[torch.arange(output.size(0)), action_batch.squeeze()]
        
        # Compute V(s_{t+1}) for all next states.
        # Expected values of actions for non_final_next_states are computed based
        # on the "older" target_net; selecting their best reward with max(1).values
        # This is merged based on the mask, such that we'll have either the expected
        # state value or 0 in case the state was final.
        next_output = torch.zeros(self.BATCH_SIZE, self.action_dim, self.num_sensitivities, device=self.device) # for 1 sensitivity 
        with torch.no_grad():
            next_inputs = torch.cat((non_final_next_states, torch.tensor([self.gammas[gamma_idx]]).unsqueeze(0).repeat(non_final_next_states.shape[0], 1)), dim=1) 
            next_output[non_final_mask] = self.target_net(next_inputs)
            action_values = torch.sum((next_output[:, :, :-1] - next_output[:, :, 1:]) * self.middle_sensitivities, dim=2)
            max_action_values = action_values.max(1).indices
            next_state_values = next_output[torch.arange(next_output.size(0)), max_action_values]        
        
        rewards_thresh = torch.nn.functional.sigmoid(self.activ_sharpness*(reward_batch.unsqueeze(-1).repeat(1, self.sensitivities.shape[0])-self.sensitivities.unsqueeze(0).repeat(reward_batch.shape[0], 1)))        
        
        assert torch.all((rewards_thresh <= 1) & (rewards_thresh >= 0)), "Rewards after activation should be between 0 or 1"
        
        # Compute the expected Q values 
        expected_state_action_values = (next_state_values * self.gammas[gamma_idx]) + rewards_thresh 
        
        # Compute Huber loss
        loss = self.criterion(state_action_values, expected_state_action_values) / float(self.num_gamma)
        self.model_loss_hist[gamma_idx].append(loss.detach().numpy())
        
        # Optimize the model
        self.optimizer.zero_grad()
        loss.backward()
        # In-place gradient clipping
        torch.nn.utils.clip_grad_value_(self.policy_net.parameters(), 100)
        self.optimizer.step()


    def select_action(self, gamma_idx):
        """
           Select action before the time horizon change
        """
        sample = random.random()
        eps_threshold = self.config.EPS_END + (self.config.EPS_START - self.config.EPS_END) * \
            math.exp(-1. * self.steps_done[gamma_idx] / self.config.EPS_DECAY)
        self.steps_done[gamma_idx] += 1

        if sample > eps_threshold:
            with torch.no_grad():
                input = torch.cat((self.states[gamma_idx], torch.tensor([self.gammas[gamma_idx]]).unsqueeze(0)), dim=1)
                
                assert self.states[gamma_idx].shape[0] == input.shape[0]
                
                z = self.policy_net(input) 
                action_values = torch.sum((z[:, :, :-1] - z[:, :, 1:]) * self.middle_sensitivities, dim=2) 
                
                assert action_values.shape[0] == self.states[gamma_idx].shape[0]
            
                # t.max(1) will return the largest column value of each row.
                # second column on max result is index of where max element was
                # found, so we pick action with the larger expected reward.
                return action_values.max(1).indices.view(1, 1)
        else:
            return torch.tensor([[self.envs[gamma_idx].action_space.sample()]], device=self.device, dtype=torch.long)
    

    def select_action_after_H_change(self, gamma_idx):
        """
           Select action after the time horizon change using the inverse Laplace transform
        """
        sample = random.random()
        eps_threshold = self.config.EPS_END + (self.config.EPS_START - self.config.EPS_END) * \
            math.exp(-1. * self.steps_done[gamma_idx] / self.config.EPS_DECAY)
        self.steps_done[gamma_idx] += 1

        if sample > eps_threshold:
            with torch.no_grad():
                input = torch.cat((self.states[gamma_idx], torch.tensor([self.gammas[gamma_idx]]).unsqueeze(0)), dim=1)
                
                assert self.states[gamma_idx].shape[0] == input.shape[0]
                Q_gamma = np.zeros((1, self.action_dim, self.num_gamma_to_tau, self.num_sensitivities))
                for idx, gamma in enumerate(self.gammas_to_tau):
                    input = torch.cat((self.states[gamma_idx], torch.tensor([gamma]).unsqueeze(0)), dim=1)
                    Q_gamma[:, :, idx, :] = self.policy_net(input) 
                
                tau_space = SVD_approximation_inverse_Laplace(self.config, Q_gamma)
                
                # eq. 13
                gammas_pow_tau = torch.tensor([self.gammas[gamma_idx]**tau for tau in range(self.K)])
                action_values = torch.matmul(torch.matmul(tau_space, self.middle_sensitivities[:-1]), gammas_pow_tau)
                
                print("Action Values shape after change in time horizon", action_values.shape)
                                    
                # t.max(1) will return the largest column value of each row.
                # second column on max result is index of where max element was
                # found, so we pick action with the larger expected reward.
                return action_values.max(1).indices.view(1, 1)
        else:
            return torch.tensor([[self.envs[gamma_idx].action_space.sample()]], device=self.device, dtype=torch.long)
      
        
    def eval_step(self, render=True):
        """
        Evaluation using the trained target network, no training involved
        :param render: whether to visualize the evaluation or not
        """
        self.max_steps = self.initial_number_steps
        
        for gamma_idx in range(self.gammas):
            
            for each_ep in range(self.config.evaluate_episodes):
                
                if self.time_horizon_change is not None and each_ep == self.time_horizon_change:
                   self.max_steps = self.later_number_steps
                
                state, info = self.envs[gamma_idx].reset(seed=self.seed) 
                state = torch.tensor(state, dtype=torch.float32, device=self.device).unsqueeze(0)

                print('Episode: {} Reward: {} Training_Max_Reward: {}'.format(each_ep, self.check_model_improved[0].item(),
                                                                                self.best_max[0].item()))
                print('-' * 64)
                self.check_model_improved = 0

                for t in count(): 
                    if (self.time_horizon_change is None) or (self.time_horizon_change is not None and each_ep < self.time_horizon_change):
                        action = self.select_action(gamma_idx)
                    else:
                        action = self.select_action_after_H_change(gamma_idx) 
                        
                    action = self.select_action(gamma_idx=-1)
                    input = torch.cat((state, torch.tensor([self.gammas[gamma_idx]]).unsqueeze(0).repeat(state.shape[0], 1)), dim=1) # NOTE evaluating for the ith gamma
                    z = self.policy_net(input)
                    observation, reward, terminated, truncated, _ = self.envs[gamma_idx].step(action.item()) 
                    reward = torch.tensor([reward], device=self.device)
                    done = terminated or truncated 

                    if render:
                        self.envs[gamma_idx].render() 
                    
                    self.check_model_improved += reward
                
                    if done or t == self.max_steps:
                        break
                    else:
                        next_state = torch.tensor(observation, dtype=torch.float32, device=self.device).unsqueeze(0)  
                        state = next_state
        print('Complete')
        
        # Calculate the runtime
        runtime = self.end_time - self.start_time
        print(f'Runtime: {runtime} seconds')

In [9]:
dqn_agent = LaplaceDQNAgentMultgamIntv(game, config)

Gammas :  tensor([0.0100, 0.8199, 0.9035, 0.9341, 0.9500])


In [10]:
dqn_agent.transition()

Episode: 0 Reward: 0 Max_Reward: 0
----------------------------------------------------------------


/Users/alpayozkan/opt/anaconda3/envs/distrl_proj/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 0 Reward: 0 Max_Reward: 0
----------------------------------------------------------------
Episode: 0 Reward: 0 Max_Reward: 0
----------------------------------------------------------------
Episode: 0 Reward: 0 Max_Reward: 0
----------------------------------------------------------------
Episode: 0 Reward: 0 Max_Reward: 0
----------------------------------------------------------------
Episode: 1 Reward: 30.0 Max_Reward: 30.0
----------------------------------------------------------------
Episode: 1 Reward: 24.0 Max_Reward: 24.0
----------------------------------------------------------------
Episode: 1 Reward: 25.0 Max_Reward: 25.0
----------------------------------------------------------------
Episode: 1 Reward: 22.0 Max_Reward: 22.0
----------------------------------------------------------------
Episode: 1 Reward: 25.0 Max_Reward: 25.0
----------------------------------------------------------------
Episode: 2 Reward: 28.0 Max_Reward: 30.0
-----------------------------

KeyboardInterrupt: 

In [ ]:
tmp = torch.rand(torch.Size([128, 5]))

In [ ]:
# take gammas
gamm_tens = tmp[:,-1]
low_gam_mask = gamm_tens < 0.5
med_gam_mask = (gamm_tens >= 0.5) &  (gamm_tens < 0.75)
high_gam_mask = (gamm_tens >= 0.75) &  (gamm_tens <= 1)

print(low_gam_mask.sum())
print(med_gam_mask.sum())
print(high_gam_mask.sum())
print(low_gam_mask.sum() + med_gam_mask.sum() + high_gam_mask.sum())

tensor(61)
tensor(30)
tensor(37)
tensor(128)


In [ ]:
res_tens = torch.zeros_like(tmp)
res_tens[low_gam_mask] = tmp[low_gam_mask]

In [ ]:
low_gam_mask

tensor([False, False,  True,  True, False, False, False, False,  True, False,
         True, False, False,  True, False, False, False, False, False, False,
         True,  True, False,  True,  True,  True,  True, False,  True, False,
         True, False,  True, False, False, False,  True, False, False,  True,
        False,  True, False, False,  True,  True, False,  True, False, False,
        False, False, False, False, False,  True,  True, False,  True, False,
         True,  True,  True, False,  True,  True,  True, False, False,  True,
         True,  True,  True, False,  True,  True, False,  True, False,  True,
        False, False,  True, False, False,  True,  True, False,  True, False,
        False, False,  True,  True, False,  True,  True,  True, False,  True,
         True,  True, False,  True, False, False, False,  True,  True,  True,
        False, False,  True, False,  True,  True, False,  True,  True, False,
        False, False, False,  True, False,  True,  True, False])

In [ ]:
res_tens

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8748, 0.5193, 0.2856, 0.4319, 0.1086],
        [0.8229, 0.2084, 0.4469, 0.1512, 0.0099],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8206, 0.9857, 0.0246, 0.2126, 0.3563],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8209, 0.3278, 0.0333, 0.2286, 0.2230],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2277, 0.5540, 0.0529, 0.8159, 0.2664],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
